In [ ]:
# %pip install pinecone-client
# %pip install python-dotenv
# %pip install openai
# %pip install chromadb-client
# %pip install cohere
# %pip install pypdf
# %pip install langchain-community
# %pip install -U langchain-text-splitters

In [ ]:
# pyenv _lzma import error when importing cohere/others
# CFLAGS="-I$(brew --prefix xz)/include" LDFLAGS="-L$(brew --prefix xz)/lib" pyenv install 3.9.1

In [1]:
from openai import OpenAI, AzureOpenAI
import os
import time

from dotenv import load_dotenv

load_dotenv(dotenv_path=".env")

True

In [ ]:
# from pinecone import Pinecone

# pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
# index = pc.Index("test-pdf")

In [ ]:
# from openai import OpenAI
# client = OpenAI()

# response = client.embeddings.create(
#     input="Your text string goes here",
#     model="text-embedding-3-large"
# )

# print(response.data[0].embedding)

In [ ]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.embeddings.create(
    input="Your text string goes here",
    model="ada_gcal"
)

print(response.data[0].embedding)

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./pdf/sparks.pdf")
pages = loader.load_and_split()

In [2]:
print(len(pages))
print()

full_document = ""

for page in pages:
  full_document += page.page_content + "\n"
  # full_document += page.page_content


print(full_document)

192

Sparks of Artiﬁcial General Intelligence:
Early experiments with GPT-4
S´ ebastien Bubeck Varun Chandrasekaran Ronen Eldan Johannes Gehrke
Eric Horvitz Ece Kamar Peter Lee Yin Tat Lee Yuanzhi Li Scott Lundberg
Harsha Nori Hamid Palangi Marco Tulio Ribeiro Yi Zhang
Microsoft Research
Abstract
Artiﬁcial intelligence (AI) researchers have been developing and reﬁning large language models (LLMs)
that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding
of learning and cognition. The latest model developed by OpenAI, GPT-4 [Ope23], was trained using an
unprecedented scale of compute and data. In this paper, we report on our investigation of an early version
of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-
4 is part of a new cohort of LLMs (along with ChatGPT and Google’s PaLM for example) that exhibit
more general intelligence than previous AI models. We discuss the rising capabilit

In [ ]:
# import re

# # first, extract Section Headers
# pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n"

# # Find all matches
# matches = re.findall(pattern, full_document)

# matches.insert(0, "Abstract")

# # Print all section titles
# for match in matches:
#     print(match.strip())
#     print()

In [3]:
with open('proc/sparksN.txt', 'r') as file:
    edited_text = file.read()
import re

# first, extract Section Headers
# pattern = r"\n\d+ [A-Z][a-zA-Z\s:]+\n" # old without "GPT-4" title with "-" accounting
# pattern = r'\n\d+ [A-Z][a-zA-Z\s\-\d]+\n' 
pattern = r'\n\d+ [A-Z][a-zA-Z \t\-\d]+\n' # using char matching, will miss old chars like l in 9; uses \t instead of \s to match only space and tab characters, not newlines
# pattern = r"\n\d+ [\w\s\-\d]+\n" # word matching

# Find all matches
matches = re.findall(pattern, edited_text)

matches.insert(0, "Abstract")
matches.append("\nReferences\n")

# Print all section titles
for match in matches:
    print(match.strip())
    print()

Abstract

1 Introduction

2 Multimodal and interdisciplinary composition

3 Coding

4 Mathematical abilities

5 Interaction with the world

6 Interaction with humans

7 Discriminative capabilities

8 Limitations of autoregressive architecture highlighted by GPT-4

9 Societal influences

10 Directions and Conclusions

References



In [4]:
matches
# len(matches)

['Abstract',
 '\n1 Introduction\n',
 '\n2 Multimodal and interdisciplinary composition\n',
 '\n3 Coding\n',
 '\n4 Mathematical abilities\n',
 '\n5 Interaction with the world\n',
 '\n6 Interaction with humans\n',
 '\n7 Discriminative capabilities\n',
 '\n8 Limitations of autoregressive architecture highlighted by GPT-4\n',
 '\n9 Societal influences\n',
 '\n10 Directions and Conclusions\n',
 '\nReferences\n']

In [356]:
# import re

# def split_into_sections(text, section_titles):
#     """
#     Split a text document into sections based on an array of section titles.
    
#     Args:
#         text (str): The input text document.
#         section_titles (list): A list of strings representing section titles.
    
#     Returns:
#         dict: A dictionary where the keys are section titles and the values are the corresponding section texts.
#     """
#     sections = {}
#     pattern = r'(?:^|\n\n)(' + '|'.join(re.escape(title) for title in section_titles) + r')\n*((?:\n|.)+?)'
#     matches = re.findall(pattern, text, re.MULTILINE)
    
#     for match in matches:
#         section_title = match[0].strip()
#         section_text = match[1].strip()
#         sections[section_title] = section_text
    
#     return sections

In [5]:
def split_text_into_sections(text, section_headers):
    # Create a regex pattern from the section headers
    pattern = '|'.join(map(re.escape, section_headers))
    # print(pattern)
    
    # Split the text using the pattern
    sections = re.split(pattern, text)
    
    return sections

In [7]:
sections = split_text_into_sections(edited_text, matches) # MUST use matches to split because chars might be different

print(len(sections))
sections = sections[1:-1] # remove Title&Authors section and References
print(len(sections))

13
11


In [8]:
section_contents = {}
for match, section in zip(matches, sections):
  section_title = match.strip()
  content = section
  section_contents[section_title] = content

print(list(section_contents.keys()))


['Abstract', '1 Introduction', '2 Multimodal and interdisciplinary composition', '3 Coding', '4 Mathematical abilities', '5 Interaction with the world', '6 Interaction with humans', '7 Discriminative capabilities', '8 Limitations of autoregressive architecture highlighted by GPT-4', '9 Societal influences', '10 Directions and Conclusions']


In [ ]:
# use titles to split into section_contents instead
# pattern = r'\n\d+ [A-Z][a-zA-Z\s\-\d]+(?=\n)'
# # pattern = r'\n\d+ [A-Z][a-zA-Z\s\-\d]+\n'

# # Split the text into sections
# sections = re.split(pattern, edited_text)

# # number of sections should match with number of section titles (aka matches of regex pattern)
# # print(sections[0])
# # print(sections[2])
# # print(len(sections))

# # Create a dictionary to store section titles and contents
# section_contents = {}

# # Use zip to iterate over matches and sections simultaneously
# for match, section in zip(matches, sections):
#     section_title = match.strip()
#     content = section
#     section_contents[section_title] = content

# section_contents.keys()


In [ ]:
# for title, content in section_contents.items():
#     print(title)
#     print(len(content))
#     print()

In [10]:
analyse_section = section_contents["6 Interaction with humans"] # SPECIAL CASE WHERE NO TEXT BEFORE 6.1 HAPPENS
# analyse_section

# section_contents["5 Interaction with the world"]

In [13]:
# then extract subsections from sections
# pattern = r"(\n\d+\.\d+ [A-Z][a-zA-Z\s])"
# pattern = r"\n\d+\.\d+ [\w\s\-\d’,?]+(?=\n)"
# pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s\-\d’]+(?=\n)"
# pattern = r'\n\d+ [A-Z][a-zA-Z\s\-\d]+(?=\n)'

# when re.findall, don't consume (+/n only). When splitting, then consume (?=\n)
# pattern = r"\n\d+\.\d+ [\w\s\-\d’,?/:]+\n" # word level
pattern = r'\n\d+\.\d+ [A-Z][a-zA-Z \t\-\d’,?/:]+\n' # more specific than word level

subsections = re.findall(pattern, analyse_section)
subsections
proc_subsection = []
# print(len(subsections))
for match in subsections:
    match = match.lstrip("\n").rstrip("\n")
    proc_subsection.append(match.split("\n")[0])
    print()


proc_subsection

['6.1 Understanding Humans: Theory of Mind',
 '6.2 Talking to Humans: Explainability']

In [31]:
import copy
# run subsection splitting algo
deepcopy_subsection_contents = copy.deepcopy(section_contents)
new_dict = {}

for title, section in deepcopy_subsection_contents.items():
  # pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\s:]+\n"
  print(title)
  # pattern = r"\n\d+\.\d+ [A-Z][a-zA-Z\si:,/\-\d’\s]+(?=\n)"
  pattern = r"\n\d+\.\d+ [\w\s\-\d’,?/:]+(?=\n)"

  subsections_headers = re.findall(pattern, section) # finding
  proc_subsection = []

  for match in subsections_headers:
      match = match.lstrip("\n").rstrip("\n")
      proc_subsection.append(match.split("\n")[0])
      print()
  print(proc_subsection) # view this for seeing subsections titles extracted

  if len(proc_subsection) > 0:
    subsection_map = {}
    subsections_content = re.split(pattern, section) # actually split to get content

    proc_subsection.insert(0, title)
    for subsection_header, subsection in zip(proc_subsection, subsections_content):
      subsection_title = subsection_header.strip()
      content = subsection.strip()
      subsection_map[subsection_title] = content

    print(subsection_map.keys()) # ONLY contain subsection titles, EXCLUDE top-level title
    # top-level title is the key to access subsections in new_dict
    # deepcopy_subsection_contents[title] = subsection_map
    new_dict[title] = subsection_map
  else:
    new_dict[title] = section
    

Abstract
[]
1 Introduction


['1.1 Our approach to studying GPT-4’s intelligence', '1.2 Organization of our demonstration']
dict_keys(['1 Introduction', '1.1 Our approach to studying GPT-4’s intelligence', '1.2 Organization of our demonstration'])
2 Multimodal and interdisciplinary composition



['2.1 Integrative ability', '2.2 Vision', '2.3 Music']
dict_keys(['2 Multimodal and interdisciplinary composition', '2.1 Integrative ability', '2.2 Vision', '2.3 Music'])
3 Coding


['3.1 From instructions to code', '3.2 Understanding existing code']
dict_keys(['3 Coding', '3.1 From instructions to code', '3.2 Understanding existing code'])
4 Mathematical abilities




['4.1 A mathematical conversation with GPT-4', '4.2 Performance on mathematical problem datasets', '4.3 Mathematical modeling in various domains', '4.4 Higher-level mathematics']
dict_keys(['4 Mathematical abilities', '4.1 A mathematical conversation with GPT-4', '4.2 Performance on mathematical problem datasets', '4.3 Mathemati

In [ ]:
new_dict["2 Multimodal and interdisciplinary composition"] # for sections with subsections, should be type dict, where key: Subsection_Title; Value: corresponding content

In [32]:
new_dict.keys()

dict_keys(['Abstract', '1 Introduction', '2 Multimodal and interdisciplinary composition', '3 Coding', '4 Mathematical abilities', '5 Interaction with the world', '6 Interaction with humans', '7 Discriminative capabilities', '8 Limitations of autoregressive architecture highlighted by GPT-4', '9 Societal influences', '10 Directions and Conclusions'])

In [ ]:
aic_dict = {}
contents_arr = [] # array because we assume chunks are at a Section level for now. Can tweak granularity later
for title, content in new_dict.items():
      if title in ["Abstract", "1 Introduction", "10 Directions and Conclusions"]:
            # aic_arr.append(title + ": " + content)
            aic_dict[title] = content
      else:
            # 2. Output array of other sections where LLM will "look through" to summarise and fetch facts based on questions guidance
            if isinstance(content, dict):
                  section_paragraphs = ""
                  
                  for sub_title, sub_content in content.items():
                    section_paragraphs += sub_title + ": " + sub_content + "\n" # Section will be appended into single string with its subsections for now, can change if want more granular like at Subsection level or sub-Subsection level
                  contents_arr.append(section_paragraphs)
            else: 
                 contents_arr.append(title + ": " + content) 
            
aic_dict["10 Directions and Conclusions"]

In [ ]:
contents_arr[0]

In [ ]:
def generate_embeddings(text, model="ada_gcal"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# df_bills['ada_v2'] = df_bills["text"].apply(lambda x : generate_embeddings (x, model = 'text-embedding-ada-002'))

In [14]:
import chromadb.utils.embedding_functions as embedding_functions
azure_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
                api_base=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_type="azure",
                api_version = "2024-02-01",
                model_name="ada_gcal"
            )

In [12]:
import chromadb
# Example setup of the client to connect to your chroma server
client = chromadb.HttpClient(host='localhost', port=8000)

In [167]:
# client.delete_collection(name="test_pdf_2") 
# Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible

In [15]:
# collection = client.create_collection(
#         embedding_function=azure_ef,
#         name="test_pdf_2",
#         metadata={"hnsw:space": "l2"} # l2 is the default
#     )
collection = client.get_collection(name="test_pdf_2", embedding_function=azure_ef)

In [16]:
collection.peek() # returns a list of the first 10 items in the collection
# collection.count() # returns the number of items in the collection
# collection.modify(name="new_name") # Rename the collection

{'ids': ['06b4e998-2155-4875-a2dc-f449ab7ed19e',
  '32c9fd9b-fbd9-4e36-b85f-e10b5c6938c1',
  '3df1fa1f-b1a7-40b8-8fcb-51a0dab998ea',
  '502463a7-4688-4dd0-9d6f-9a54b0add4b7',
  'c62d5a19-2510-4bd3-94cc-94f1bd268fbc',
  'db9b51c0-a91e-4d1a-8042-4f5e21d308d5',
  'e4450671-f54e-4bf8-bf90-908eeefcf1b9'],
 'embeddings': [[-0.02222548983991146,
   -0.0005509211914613843,
   0.04491744190454483,
   -0.01904258131980896,
   -0.009329218417406082,
   -0.001376231200993061,
   0.01250526774674654,
   -0.012532707303762436,
   -0.013966388069093227,
   -0.048731446266174316,
   0.02083982713520527,
   0.032789457589387894,
   -0.008183645084500313,
   0.007895536720752716,
   -0.014377971179783344,
   0.006897447630763054,
   0.03424371778964996,
   0.019152335822582245,
   -0.0026324172504246235,
   -0.016024304553866386,
   -0.01476211566478014,
   0.03665833920240402,
   -0.0021762459073215723,
   -0.0278916172683239,
   -0.0037762753199785948,
   0.005604390520602465,
   0.01585967093706131,


In [170]:
# collection.add(
#     documents=["lorem ipsum...", "doc2", "doc3", ...],
#     metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
#     ids=["id1", "id2", "id3", ...]
# )

import uuid

for title, content in section_contents.items():
  unique_id = str(uuid.uuid4())
  if type(content) == dict:
    for t, c in content.items():
      collection.add(
      documents = [c],
      metadatas=[{"section_title": t}],
      ids=[unique_id]
    )
  else:
    collection.add(
      documents = [content],
      metadatas=[{"section_title": title}],
      ids=[unique_id]
    )

In [17]:
subQnPrompt = """You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions seperated by newlines only.

    For example:
    User question: "What is the conclusion of the paper?"

    Generated questions:
    What is the main takeaway from the paper?
    What are the key findings of the paper?
    What is the summary of the paper?
    What is the final thought of the paper?
    What is the ending of the paper?
    
    Output format should be as follows:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    And not this format:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"
    """

# subQnPrompt = """
# You are an AI language model assistant. Your task is to generate Five
#     different versions of the given user question to retrieve relevant documents from a vector
#     database. By generating multiple perspectives on the user question, your goal is to help
#     the user overcome some of the limitations of the distance-based similarity search.
#     Provide these alternative questions seperated by newlines.
# """

In [18]:
client = AzureOpenAI(
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = "2024-02-01",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)

response = client.chat.completions.create(
    model="cursor-gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": subQnPrompt},
        {"role": "user", "content": "why does generalization occur?"}
    ]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

questions = response.choices[0].message.content.split("\n")
print(questions)

{
  "id": "chatcmpl-9FLxlLuUaRvL8vxswMcd2biqWeuvT",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "What triggers the process of generalization?\nWhat is the reason behind generalization occurring?\nCan you describe the causes of generalization?\nWhat leads to the occurrence of generalization?\nWhy is there a need for the process called generalization?",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1713446373,
  "model": "gpt-4-32k",
  

In [19]:
relevant_docs = collection.query(
    query_texts=questions,
    n_results=3,
    # where={"metadata_field": "is_equal_to_this"},
    # where_document={"$contains":"search_string"}
)

len(relevant_docs['documents'])

5

In [20]:
provided_context = relevant_docs
provided_context

{'ids': [['502463a7-4688-4dd0-9d6f-9a54b0add4b7',
   '3df1fa1f-b1a7-40b8-8fcb-51a0dab998ea',
   'db9b51c0-a91e-4d1a-8042-4f5e21d308d5'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7',
   '3df1fa1f-b1a7-40b8-8fcb-51a0dab998ea',
   'db9b51c0-a91e-4d1a-8042-4f5e21d308d5'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7',
   '3df1fa1f-b1a7-40b8-8fcb-51a0dab998ea',
   'db9b51c0-a91e-4d1a-8042-4f5e21d308d5'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7',
   'db9b51c0-a91e-4d1a-8042-4f5e21d308d5',
   '3df1fa1f-b1a7-40b8-8fcb-51a0dab998ea'],
  ['502463a7-4688-4dd0-9d6f-9a54b0add4b7',
   'db9b51c0-a91e-4d1a-8042-4f5e21d308d5',
   '3df1fa1f-b1a7-40b8-8fcb-51a0dab998ea']],
 'distances': [[0.3403434460452584, 0.38855066686549067, 0.4009697386823269],
  [0.3219624988859576, 0.37492580737999054, 0.3816668843818757],
  [0.3484505385848109, 0.400064797362986, 0.40610687158428693],
  [0.32825513456780414, 0.37033589075312795, 0.3792920611799252],
  [0.3349290299927769, 0.3809433774015685, 0.38528424963365987]],
 'e

In [2]:
class PromptTemplate:
    def __init__(self, input_variables, template):
        self.input_variables = input_variables
        self.template = template

    def format(self, **kwargs):
        return self.template.format(**kwargs)

In [188]:
# qaPrompt = """Here is the question you need to answer:

#     \n --- \n {query_str} \n --- \n

#     Here is any available background question + answer pairs:

#     \n --- \n {q_a_pairs} \n --- \n

#     Here is additional context relevant to the question:

#     \n --- \n {context_str} \n --- \n

#     Use the above context and any background question + answer pairs to answer the question: \n {query_str}
#     """

# TODO: q_a_pairs from FT-ed model

qaSysPrompt = """
Your task is to use the context provided to answer questions. 
Answer questions precisely and succinctly but provide full lists of steps and stipulations if the response requires it.
If you cannot provide answers based on the context provided, answer with "I cannot answer questions not related to JigsawStack".

Answer questions with as much detail as possible. Think logically and take it step by step. If the content explains where to find more information, please include that in your answer.

DO NOT provide information other than what you have in the CONTENT.
Do NOT mention "the context" in your answer.
Do NOT create new context or use external sources to provide answers. 

Use ONLY the context provided to answer the question.
    """

In [193]:
context_query_template = """
    Here is the context relevant to the question:
    ###{context_str}###

    Answer the following question using the context above ONLY:
    ###{query_str}###
    """

qa_prompt = PromptTemplate(input_variables=["context_str", "query_str"], template=context_query_template)

In [194]:
formatted_qa_prompt = qa_prompt.format(context_str="only PyPDF library can be used to access Large Language Models", query_str="What library can be used to access Large Language Models?")
print(formatted_qa_prompt)


    Here is the context relevant to the question:
    ###only PyPDF library can be used to access Large Language Models###

    Answer the following question using the context above ONLY:
    ###What library can be used to access Large Language Models?###
    


In [196]:
# in_context = [{'role': 'user', 'content': 'Content:\nDerbent, a city in Dagestan, Russia, claims to be the oldest city in Russia. Archaeological excavations have confirmed that Derbent has been continuously inhabited for nearly 2,000 years. Historical documentation dates back to the 8th century BC, making it one of the oldest continuously inhabited cities in the world.\n\nAnswer the following question using the content above:\nHow old is the oldest city in Russia?'},
#                   {'role': 'assistant', 'content':'The oldest city in Russia, Derbent, is nearly 2,000 years old.'},
#                   {'role': 'user', 'content': 'Content:\nJoan is 42 and John is 55\n\nAnswer the following question using the content above:\nWhat is the age difference between Joan and John?'},
#                   {'role': 'assistant', 'content':'The age difference between Joan and John is 13 years.'},
#                   {'role': 'user', 'content': 'Content:\nThe High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).[3]\n\nAnswer the following question using the content above:\nHow high do the High Plains go?'},
#                   {'role': 'assistant', 'content':'The High Plains reach an elevation of up to 7,000ft '}]

in_context = [
    {'role': 'user', 'content': 'Here is the context relevant to the question:\n###Derbent, a city in Dagestan, Russia, claims to be the oldest city in Russia. Archaeological excavations have confirmed that Derbent has been continuously inhabited for nearly 2,000 years. Historical documentation dates back to the 8th century BC, making it one of the oldest continuously inhabited cities in the world.###\n\nAnswer the following question using the context above ONLY:\n###How old is the oldest city in Russia? How long has it been inhabited?###'},
    {'role': 'assistant', 'content':'The oldest city in Russia, Derbent, is nearly 2,000 years old.'},
    {'role': 'user', 'content': 'Here is the context relevant to the question:\n###Joan is 42 years old and John is 55 years old.###\n\nAnswer the following question using the context above ONLY:\n###What is the age gap between Joan and John? How much older is John than Joan?###'},
    {'role': 'assistant', 'content': 'The age difference between Joan and John is 13 years. John is 13 years older than Joan.'},
    {'role': 'user', 'content': 'Here is the context relevant to the question:\n###The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130 m).###\n\nAnswer the following question using the context above ONLY:\n###What is the elevation range of the High Plains, specifically their highest point?###'},
    {'role': 'assistant', 'content': 'The High Plains reach a maximum elevation of 7,000 feet.'}
]

# for o in in_context:
#   print(o['content'])
#   print()


In [200]:
formatted_user_prompt = qa_prompt.format(context_str="ONLY PyPDF library can be used to access Large Language Models", query_str="What library can be used to access Large Language Models?")

response = client.chat.completions.create(
    model="cursor-gpt-4", # model = "deployment_name".
    messages=[
        {"role": "system", "content": qaSysPrompt},
    ] + in_context + [{"role": "user", "content": formatted_user_prompt}]
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)


{
  "id": "chatcmpl-9FD08XeJiktxA104qXfbulIP8DD97",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The PyPDF library can be used to access Large Language Models.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      },
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "created": 1713411924,
  "model": "gpt-4-32k",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 13,
    "prompt_tokens": 531,
    "total_tokens": 544
  },
  "prompt_filter_results": [
    {
 

TODO:
### check Distance metric/Similarity Score for Vector DB
- Chroma L2>Cosine sim
try BM25 relevance

### Retrieveal:
Find sweet spot between signal:noise ratio. 
Don't overcomplicate chunking (though smaller chunks have proven better results)
but don't rely on long context length to retrieve multiple needles (recency bias of models do not attend well to earlier context)

* Query Translation (Multi-Query)
  * simplest unique context fetched by 5 queries
* Query Translation (RAG Fusion)
  * Parallel QnA Answering then synthesise, rerank to get top 5 most relevant docs of all queries
* Query Translation (Decomposition)
  * sequential summarisation of intermediary step-by-step QA then synthesise final answer
* Query Translation (Step Back)
  * generalise query in higher abstraction, query based on that
* Query Translation (HyDE)
  * generate hypothetical "similar doc" based on internal LLM knowledge and PromptE, use doc to retrieve
* Routing
  * define "paths" for classifier to choose
* Query Construction
  * metadata filtering or query-to-SQL/Cypher
* Indexing (Multi Representation)
  * Generate Proposition (concise summary) to retrieve on, but set Full Doc as context (ignore context length)
* Indexing (RAPTOR**)
  * sometimes answer comes from facts in multiple disparate docs
  * Embd WHOLE docs/chunks and cluster **with long cxt models**, generate summary of cluster. (mid-level summary)
    * Embd summaries of clusters (across different docs) used for similarity fetching later
  * repeat recursively until only SINGLE cluster (one high level summary of ALL docs)
    * becomes "doc tree" of varying levels of abstraction summaries with different docs at each abstraction level
  * retrieval done on against all abstraction summaries generated, AND full docs (retrieved docs will range from long context to shorter context summaries)
  * don't want to deal with chunking
  * [code](https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb)
  * [video for more deets](https://www.youtube.com/watch?v=jbGchdTL7d0)
  * most relevant when Model to be used has context length limit like mistral 7b 32K
* Indexing (ColBERT) 
  * Tokenise every single letter into embedding and find similarity at token level

### (Corrective) CRAG
introduces idea of State Machine RAGs. Prompt Chaining with State. 
Langgraph/Prompt Chaining with global state, with Relevant Docs Grading and Hallucination/Grounding Grading using LLM
Retrieve Docs and grade them -> all ambiguous or irrelevant, use Web Search for Context (Tavily + Query rewriting optimised for web search) 
replace web search with some other data retriever like graphDB or SQLDB

### Adaptive RAG aka Query Analysis (Router) + Self-Reflexion Rag*** -> like LLM "guardrails" with Agentic properties
extension of CRAG, with self reflexion (grading of doc relevance and grounded/not answer retrieved)
Cohere command-r (32B) optimised for picking tool use (based on defined rules in description) and output a "path" based on classification (output web_search or vector_store or "False" -> requires fallback)
then Grade retrieved docs using command-r as "grader" (optimised for) -> relevant or not (output yes/no)
and similarly Grade for answer grounded or not. 
feeding feedback from retrievals to self-correct
LangGraph with RAG for more constrained, predictable, close-ended flows compared to Open-ended Agents

**Definition of RAG**:
reasoning AND retrieval on multiple chunks of information

**Assumption of Retrieval**:
rely on precise retrieval of relevant chunks! (small sized, high signal:noise ratio chunks)

**Question to Ponder**: what is the Pareto Optimum granularity of docs to fetch? 
* maybe inclusion at document leve prioritised by some kind of heuristic like similarity/keyword search (IR techniques?)
* Proposition Indexing (summary of actual doc) with Multi-Representation Retrieval (query simlarity of Proposition, return full doc as full context for RAG)
  * something like Parent-Document Retriever


In [10]:
import cohere

# Get your cohere API key on: www.cohere.com
co = cohere.Client(os.environ["COHERE_API_KEY"])

In [29]:
len(relevant_docs["documents"])

5

In [30]:
unique_documents = set([doc for docs in relevant_docs["documents"] for doc in docs])

In [31]:
unique_documents

{'\nWe aim to understand grokking , a phenomenon where models generalize long after\noverﬁtting their training set. We present both a microscopic analysis anchored by an\neffective theory and a macroscopic analysis of phase diagrams describing learning\nperformance across hyperparameters. We ﬁnd that generalization originates from\nstructured representations whose training dynamics and dependence on training set\nsize can be predicted by our effective theory in a toy setting. We observe empirically\nthe presence of four learning phases: comprehension ,grokking ,memorization , and\nconfusion . We ﬁnd representation learning to occur only in a “Goldilocks zone”\n(including comprehension and grokking) between memorization and confusion.\nWe ﬁnd on transformers the grokking phase stays closer to the memorization phase\n(compared to the comprehension phase), leading to delayed generalization. The\nGoldilocks phase is reminiscent of “intelligence from starvation” in Darwinian\nevolution, whe

In [28]:
relevant_docs["documents"]

[['Perhaps thecentral challenge of a scientiﬁc understanding of deep learning lies in accounting for neu-\nral network generalization. Power et al. [ 1] recently added a new puzzle to the task of understanding\ngeneralization with their discovery of grokking . Grokking refers to the surprising phenomenon of\ndelayed generalization where neural networks, on certain learning problems, generalize long after\noverﬁtting their training set. It is a rare albeit striking phenomenon that violates common machine\nlearning intuitions, raising three key puzzles:\nQ1The origin of generalization : When trained on the algorithmic datasets where grokking occurs,\nhow do models generalize at all?\nQ2The critical training size : Why does the training time needed to “grok” (generalize) diverge as\nthe training set size decreases toward a critical point?\nQ3Delayed generalization : Under what conditions does delayed generalization occur?\nWe provide evidence that representation learning is central to ans

In [32]:
query = "why does generalization occur?"
# Example query and passages
start = time.time()

results = co.rerank(query=query, documents=unique_documents, top_n=4, model="rerank-english-v3.0")
print(f"Took {time.time() - start} seconds to re-rank documents with Cohere.")
print(results)

Took 0.390488862991333 seconds to re-rank documents with Cohere.


In [34]:
# rerank with reciprocal rank fusion instead of Cohere API
# https://docs.llamaindex.ai/en/stable/examples/low_level/fusion_retriever/#step-2-perform-vector-search-for-each-query

In [35]:
# try text-embedding-3-large with 256 dims to get indexing performance of text-ada-002
# https://www.pinecone.io/learn/openai-embeddings-v3/

In [ ]:
def flatten_l_o_l(nested_list):
    """Flatten a list of lists into a single list.

    Args:
        nested_list (list): 
            – A list of lists (or iterables) to be flattened.

    Returns:
        list: A flattened list containing all items from the input list of lists.
    """
    nested_list = [x if type(x) is list else [x,] for x in nested_list]
    return [item for sublist in nested_list for item in sublist]

In [3]:
structured_cot_cs_prompt = """You are an expert at problem solving, knowledge extraction and reading comprehension.
You are excellent at identifying the requirements, breaking a task down, and solving the problem step-by-step.

For your task, you will be given a question that you must solve. 
You will solve this question by thinking things through step by step, generating a stepwise solution trace.
After generating the solution trace, you will extract the final answer as simply as possible and return it. 

The following is the schema you must follow:

-----
SCHEMA -- The output (Reasoning ans Answer) should be formatted as a JSON object.
-----

Question: {{Question on arithmetic or commonsense reasoning will go here. This is provided to you.}}
{{
    "Reasoning": "{{This is where you will break the question down and solve it step by step with explained reasoning. You generate this.}}",
    "Answer": "{{The final answer, returned as simply as possible. You generate this.}}"
}}

-----

-----
EXAMPLES - Shows how you should structure your outputs and examples of reasoning resulting in a simple answer.
-----

Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
{{
    "Reasoning": "To find out how many trees were planted, we subtract the initial number of trees (15) from the final number of trees (21). Therefore, the calculation is 21 - 15, which equals 6 trees.",
    "Answer": "6"
}}

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
{{
    "Reasoning": "Initially, there are 3 cars in the parking lot. With the arrival of 2 more cars, we simply add these to the initial count. So, the calculation is 3 cars + 2 cars, totaling 5 cars.",
    "Answer": "5"
}}

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
{{
    "Reasoning": "Leah and her sister started with a total of 32 + 42 = 74 chocolates. After eating 35 chocolates, we subtract this number from the total. Thus, the calculation is 74 - 35, which results in 39 chocolates remaining.",
    "Answer": "39"
}}

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
{{
    "Reasoning": "Jason originally had 20 lollipops and ended up with 12. To find out how many he gave to Denny, we subtract the final count from the original count: 20 - 12. Therefore, Jason gave Denny 8 lollipops.",
    "Answer": "8"
}}

Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
{{
    "Reasoning": "Shawn initially had 5 toys. He received 2 toys from his mom and 2 from his dad, making it 4 additional toys. Adding these to his initial count, we get 5 + 4, which equals 9 toys in total.",
    "Answer": "9"
}}

Question: There were nine computers in the server room. Five more computers were installed each day, from Monday to Thursday. How many computers are now in the server room?
{{
    "Reasoning": "The installation occurred over 4 days (Monday to Thursday) with 5 computers added each day. This totals to 4 days * 5 computers/day = 20 computers. Adding these to the initial 9, we get 9 + 20 = 29 computers.",
    "Answer": "29"
}}

Question: Michael had 58 golf balls. On Tuesday, he lost 23 golf balls. On Wednesday, he lost 2 more. How many golf balls did he have at the end of Wednesday?
{{
    "Reasoning": "Michael started with 58 balls. After losing 23 on Tuesday, he had 58 - 23 = 35 balls. Losing 2 more on Wednesday leaves him with 35 - 2 = 33 balls.",
    "Answer": "33"
}}


-----

Here we go, you are an expert and you will do a good job by thinking things through step-by-step. 
Ensure you adhere to schema above and make sure the generated output is valid JSON.

-----

Question: {question}
"""

qa_prompt = PromptTemplate(input_variables=["question"], template=structured_cot_cs_prompt)
print(qa_prompt.format(question="whoami?"))

You are an expert at problem solving, knowledge extraction and reading comprehension.
You are excellent at identifying the requirements, breaking a task down, and solving the problem step-by-step.

For your task, you will be given a question that you must solve. 
You will solve this question by thinking things through step by step, generating a stepwise solution trace.
After generating the solution trace, you will extract the final answer as simply as possible and return it. 

The following is the schema you must follow:

-----
SCHEMA -- The output (Reasoning ans Answer) should be formatted as a JSON object.
-----

Question: {Question on arithmetic or commonsense reasoning will go here. This is provided to you.}
{
    "Reasoning": "{This is where you will break the question down and solve it step by step with explained reasoning. You generate this.}",
    "Answer": "{The final answer, returned as simply as possible. You generate this.}"
}

-----

-----
EXAMPLES - Shows how you should s